In [ ]:
# Option 1: Install the full training bundle (Recommended)
!pip install "mlx-lm[train]"

# Option 2: Just install the missing library
!pip install datasets

In [1]:
!python -c "import datasets; print('Datasets library found!')"

Datasets library found!


In [4]:
!python -m mlx_lm.lora \
  --model mlx-community/gemma-2-2b-it-4bit \
  --train \
  --data ./anime_data \
  --iters 500 \
  --batch-size 1 \
  --learning-rate 1e-5 \
  --adapter-path ./anime_adapters

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Calling `python -m mlx_lm.lora...` directly is deprecated. Use `mlx_lm.lora...` or `python -m mlx_lm lora ...` instead.
Loading pretrained model
Fetching 7 files: 100%|███████████████████████| 7/7 [00:00<00:00, 112923.57it/s]
Loading datasets
Training
Trainable parameters: 0.038% (0.983M/2614.342M)
Starting training..., iters: 500
Calculating loss...: 100%|████████████████████████| 2/2 [00:01<00:00,  1.14it/s]
Iter 1: Val loss 6.137, Val took 1.754s
Iter 10: Train loss 4.610, Learning Rate 1.000e-05, It/sec 1.076, Tokens/sec 82.758, Trained Tokens 769, Peak mem 2.301 GB
Iter 20: Train loss 3.438, Learning Rate 1.000e-05, It/sec 1.444, Tokens/sec 111.037, Trained Tokens 1538, Peak mem 2.301 GB
Iter 30: Train loss 2.651, Learning Rate 1.000e-05, It/sec 1.427, Tokens/sec 109.771, Trained Tokens 2307, Peak mem 2.301 GB
Iter 40: Train loss 2.060, Learning Rate 1.000e-05, It/sec 1.428, Tokens/sec 109.829, Trained Tokens 3076, Peak mem 2.301 GB
Iter 50: Train loss 1.514, Learning Rate 1.000e-

In [ ]:
from mlx_lm import load, generate

# 1. Load the model (the 4-bit version is essential for M1 RAM efficiency)
model_name = "mlx-community/gemma-2-2b-it-4bit"
model, tokenizer = load(model_name, adapter_path="./anime_adapters")

# 2. Initialize an empty conversation history
messages = []

def chat_with_memory():
    print("--- Gemma (Fine-tuned) Chatbot (Type 'clear' to reset or 'exit' to stop) ---")
    
    while True:
        # What's Rudo's vital instrument in anime Gachiakuta?
        # Who are the main characters of GXYZ123?
        user_input = input("\nYou: ")
        
        if user_input.lower() == 'exit':
            break
        if user_input.lower() == 'clear':
            messages.clear()
            print("History cleared!")
            continue

        # Add user message to history
        messages.append({"role": "user", "content": user_input})

        # Format history into the Gemma-specific string format
        # This adds the <start_of_turn> and <end_of_turn> tags automatically
        prompt = tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )

        # Generate and stream the response
        print("\nGemma (Fine-tuned): ", end="", flush=True)
        response = generate(
            model, 
            tokenizer, 
            prompt=prompt, 
            max_tokens=1000, 
            verbose=True # Streaming enabled
        )
        
        # Save the model's response to history for next time
        messages.append({"role": "assistant", "content": response})

# Run the chat
chat_with_memory()

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

--- Gemma (Fine-tuned) Chatbot (Type 'clear' to reset or 'exit' to stop) ---



You:  Who are the main characters of GXYZ123?



==========e-tuned): 
The primary characters include 'Protocol-X,' a sentient glitch trying to restore its memory, and 'User-774,' an anonymous moderator who discovers a hidden world within the site's code.
Prompt: 21 tokens, 16.293 tokens-per-sec
Generation: 42 tokens, 32.357 tokens-per-sec
Peak memory: 4.484 GB
